In [1]:
import pandas as pd
import sapdi
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.layers import Dropout
import numpy as np
np.random.seed(20)

ws = sapdi.get_workspace(name='covid_connection')
dc = ws.get_datacollection(name='datasets')
with dc.open('GCP_IN_data.csv').get_reader() as reader:
    df_all_states = pd.read_csv(reader)

In [2]:
# pip install tensorflow==2.1.0

In [3]:
df_all_states.Date = [x[2:] for x in df_all_states.Date]
df_all_states.Date = pd.to_datetime(df_all_states.Date,dayfirst=True)
kerala = df_all_states[df_all_states['State/UnionTerritory']=='Kerala']
karnataka = df_all_states[df_all_states['State/UnionTerritory']=='Karnataka']
Maharashtra = df_all_states[df_all_states['State/UnionTerritory']=='Maharashtra']

In [4]:
from sklearn.preprocessing import StandardScaler

def scale_n_transform(train_data):
    
    scaler = StandardScaler()
    scaler.fit(train_data)
    scaled_train_data = scaler.transform(train_data)
    return scaled_train_data ,scaler

In [5]:
df_all_states = df_all_states.set_index('Date')

In [6]:
n_input = 15
n_features= 1

def build_model():
    model = Sequential()
    model.add(GRU(200, activation='relu', return_sequences=True, input_shape=(n_input, n_features)))
    model.add(Dropout(rate=0.2))
    model.add(GRU(100, activation='relu'))
#     model.add(Dropout(rate=0.4))
#     model.add(LSTM(100,activation='relu',return_sequences=True))
#     model.add(Dropout(rate=0.5))
#     model.add(LSTM(50,activation='relu'))
#     model.add(Dropout(rate=0.2))
#     model.add(Dense(50))
#     model.add(Dropout(rate=0.4))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [7]:
def train_model(model,scaled_train_data,n_epochs):
    generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)
    model.fit_generator(generator,epochs=n_epochs)
    return model

In [8]:
def generate_future_timestamps(df,num_prediction):
    last_date = df.index[-1]
    prediction_dates = pd.date_range(last_date,freq='D', periods=num_prediction+1).tolist()
    return prediction_dates

In [9]:
forecast_dates = generate_future_timestamps(df_all_states,15)

In [10]:
final_forecasts = pd.DataFrame()

for state in [karnataka,Maharashtra]:
    state_name = state.iloc[1]['State/UnionTerritory']
    print(f'processing for state : {state_name}')
    
    df_forecast = pd.DataFrame()
    df_forecast['Date'] = forecast_dates[1:]
    df_forecast['State/UnionTerritory'] = state_name
    
    for column in ['Cured','Deaths','Confirmed']:
        
        print(f'Forecasting for {state_name} -- {column}:')
        temp_cases_df = state[['Date',column]]
        temp_cases_df = temp_cases_df.set_index("Date")
        scaled_train_data ,scaler = scale_n_transform(temp_cases_df)
        
        # start training
        model = build_model()
        model  = train_model(model,scaled_train_data,8)
        
        lstm_predictions_scaled = []
        batch = scaled_train_data[-n_input:]
        current_batch = batch.reshape((1, n_input, n_features))

        for i in range(30):   
            lstm_pred = model.predict(current_batch)[0]
            lstm_predictions_scaled.append(lstm_pred) 
            current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

        lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
        print(f'df_forecast: {len(df_forecast)}')
        print(len(lstm_predictions[n_input:]))
        print(lstm_predictions[n_input:])
        df_forecast[column] = lstm_predictions[n_input:]
        
#     print( df_forecast.head())   
    final_forecasts = pd.concat([final_forecasts,df_forecast])


processing for state : Karnataka
Forecasting for Karnataka -- Cured:
Train for 37 steps
Epoch 1/8
37/37 [==============================] - 8s 229ms/step - loss: 0.3309
Epoch 2/8
37/37 [==============================] - 4s 114ms/step - loss: 0.0610
Epoch 3/8
37/37 [==============================] - 4s 116ms/step - loss: 0.0089
Epoch 4/8
37/37 [==============================] - 4s 113ms/step - loss: 0.0153
Epoch 5/8
37/37 [==============================] - 4s 111ms/step - loss: 0.0102
Epoch 6/8
37/37 [==============================] - 4s 113ms/step - loss: 0.0112
Epoch 7/8
37/37 [==============================] - 4s 111ms/step - loss: 0.0163
Epoch 8/8
37/37 [==============================] - 4s 111ms/step - loss: 0.0306
df_forecast: 15
15
[[326.1656 ]
 [335.62448]
 [345.30316]
 [355.21704]
 [365.37756]
 [375.7961 ]
 [386.4834 ]
 [397.4509 ]
 [408.7101 ]
 [420.27313]
 [432.15253]
 [444.3605 ]
 [456.91016]
 [469.81454]
 [483.08676]]
Forecasting for Karnataka -- Deaths:
Train for 37 steps
E

In [11]:
final_forecasts

,Date,State/UnionTerritory,Cured,Deaths,Confirmed
0,2020-04-30,Karnataka,326.165588,25.119984,630.448425
1,2020-05-01,Karnataka,335.624481,25.318213,634.967712
2,2020-05-02,Karnataka,345.303162,25.507177,639.294678
3,2020-05-03,Karnataka,355.217041,25.687294,643.438965
4,2020-05-04,Karnataka,365.377563,25.858974,647.408264
5,2020-05-05,Karnataka,375.796112,26.022654,651.209595
6,2020-05-06,Karnataka,386.483398,26.178740,654.849976
7,2020-05-07,Karnataka,397.450897,26.327532,658.335449
8,2020-05-08,Karnataka,408.710114,26.469366,661.672546
9,2020-05-09,Karnataka,420.273132,26.604570,664.867065


In [12]:
final_forecasts['active_cases'] = (final_forecasts.Confirmed - ( final_forecasts.Cured + final_forecasts.Deaths ))

In [13]:
final_forecasts

,Date,State/UnionTerritory,Cured,Deaths,Confirmed,active_cases
0,2020-04-30,Karnataka,326.165588,25.119984,630.448425,279.162842
1,2020-05-01,Karnataka,335.624481,25.318213,634.967712,274.025024
2,2020-05-02,Karnataka,345.303162,25.507177,639.294678,268.484344
3,2020-05-03,Karnataka,355.217041,25.687294,643.438965,262.534637
4,2020-05-04,Karnataka,365.377563,25.858974,647.408264,256.171722
5,2020-05-05,Karnataka,375.796112,26.022654,651.209595,249.390839
6,2020-05-06,Karnataka,386.483398,26.178740,654.849976,242.187836
7,2020-05-07,Karnataka,397.450897,26.327532,658.335449,234.557007
8,2020-05-08,Karnataka,408.710114,26.469366,661.672546,226.493073
9,2020-05-09,Karnataka,420.273132,26.604570,664.867065,217.989349


In [14]:
Maharashtra

,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
76,2020-03-09,6:00 PM,Maharashtra,2,0,0,0,2
91,2020-03-10,6:00 PM,Maharashtra,5,0,0,0,5
97,2020-03-11,6:00 PM,Maharashtra,2,0,0,0,2
120,2020-03-12,6:00 PM,Maharashtra,11,0,0,0,11
133,2020-03-13,6:00 PM,Maharashtra,14,0,0,0,14
146,2020-03-14,6:00 PM,Maharashtra,14,0,0,0,14
153,2020-03-15,6:00 PM,Maharashtra,32,0,0,0,32
167,2020-03-16,6:00 PM,Maharashtra,32,0,0,0,32
182,2020-03-17,6:00 PM,Maharashtra,36,3,0,1,39
197,2020-03-18,6:00 PM,Maharashtra,39,3,0,1,42
